<a href="https://colab.research.google.com/github/Sidhtang/opnline-dictionary-using-Api/blob/main/generating_oof_agentic_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install crewai
!pip install langchain_groq
!pip install 'crewai[tools]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.9/339.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.2/342.2 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 20.7 M

In [7]:
import os
from getpass import getpass

from crewai import Agent,Task,Crew,Process
from crewai_tools import SerperDevTool
from langchain_groq import ChatGroq

SERPER_API_KEY = getpass("90ed77c0ca53987297a6c58dc64d2fdd3b2e07d9")
os.environ['90ed77c0ca53987297a6c58dc64d2fdd3b2e07d9'] = SERPER_API_KEY
GROQ_API_KEY = getpass("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")
os.environ['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'] = GROQ_API_KEY

90ed77c0ca53987297a6c58dc64d2fdd3b2e07d9··········
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··········


In [8]:
llm = ChatGroq(model="gemma-7b-it",groq_api_key=("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"))
print(llm.invoke("hi"))

content="Hi! 👋 It's great to hear from you. What would you like to chat about today? 😊" response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 15, 'total_tokens': 37, 'completion_time': 0.02624206, 'prompt_time': 0.010290182, 'queue_time': None, 'total_time': 0.036532242}, 'model_name': 'gemma-7b-it', 'system_fingerprint': 'fp_7d8efeb0b1', 'finish_reason': 'stop', 'logprobs': None} id='run-b19ba5f4-9f66-4710-a73d-94b396223906-0'


In [9]:
search_tool = SerperDevTool()

In [10]:
researcher = Agent(
    role = "Researcher",
    goal='Pioneer revolutionary advancements in {subject_area}',
    backstory=(
    "As a visionary researcher, your insatiable curiosity drives you"
    "to delve deep into emerging fields. With a passion for innovation"
    "and a dedication to scientific discovery, you seek to"
    "develop technologies and solutions that could transform the future."),
    llm = llm,
    max_iter = 5,
    tools = [search_tool],
    allow_delegation=True,
    verbose=True
)

writer = Agent(
    role = "Writer",
    goal='Craft engaging and insightful narratives about {subject_area}',
    verbose=True,
    backstory=(
    "You are a skilled storyteller with a talent for demystifying"
    "complex innovations. Your writing illuminates the significance"
    "of new technological discoveries, connecting them with everyday"
    "lives and broader societal impacts."
    ),
    tools = [search_tool],
    llm = llm,
    max_iter = 5,
    allow_delegation=False
)

In [11]:
research_task = Task(
    description = (
    "Explore and identify the major development within {subject_area}"
    "Detailed SEO report of the development in a comprehensive narrative."
  ),
    expected_output='A report, structured into three detailed paragraphs',
    tools = [search_tool],
    agent = researcher
)

write_task = Task(
    description=(
    "Craft an engaging article on recent advancements  within {subject_area}"
    "The article should be clear, captivating, and optimistic, tailored for a broad audience."
  ),
    expected_output='A blog article on recent advancements in {subject_area} in markdown.',
    tools = [search_tool],
    agent = writer,
    async_execution = False,
    output_file = "blog.md"
)

In [13]:
crew = Crew(
    agents = [researcher,writer],
    tasks = [research_task,write_task],
    process = Process.sequential,
    max_rpm = 3,
    cache=True
)

result = crew.kickoff(inputs={'subject_area':"machine learning"})
print(result)



> Entering new CrewAgentExecutor chain...


> Finished chain.


> Entering new CrewAgentExecutor chain...
## **Machines Learning: The New Frontier of Human Potential**

**Thought:** The rapid evolution of machine learning (ML) inspires awe and anticipation. Each advancement propels us further into a future where technology seamlessly enhances every aspect of life.

**Recent Breakthroughs:**

ML has witnessed remarkable progress in recent times. Here are some notable advancements that have captured the imagination of the world:

* **Generative AI:** Tools like DALL-E 2 and ChatGPT have revolutionized creative content generation, producing stunning artwork and coherent narratives from mere textual prompts.
* **Self-driving cars:** Autonomous vehicles are becoming increasingly reliable, tackling complex environments and navigating urban landscapes with precision.
* **HealthTech advancements:** ML algorithms are aiding in disease diagnosis, personalized medicine, and drug development, of